# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

## Imports

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU'), tf.__version__

2025-02-11 14:41:37.908351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739284897.916353    2015 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739284897.918682    2015 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 14:41:37.930790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

In [ ]:

from dotenv import load_dotenv
from tensorflow import keras
import cv2

2025-02-12 07:34:35.926453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739345675.938305    9487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739345675.941706    9487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 07:34:35.954963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Download datasets

In [2]:
# Loading kaggle keys
load_dotenv()

True

In [3]:
%load_ext dotenv
%dotenv

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


## Loading Images

In [6]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

class_mapping = dict(zip(range(len(CLASS_NAMES)), CLASS_NAMES))
class_mapping

{0: 'elbow positive',
 1: 'fingers positive',
 2: 'forearm fracture',
 3: 'humerus fracture',
 4: 'humerus',
 5: 'shoulder fracture',
 6: 'wrist positive'}

In [ ]:
from pathlib import Path

TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)

TRAIN_IMAGE_DIR = TRAIN_DIR / 'images'
TRAIN_LABELS_DIR = TRAIN_DIR / 'labels'

VALID_IMAGE_DIR = VALIDATION_DIR/'images'
VALID_LABELS_DIR = VALIDATION_DIR/'labels'

TEST_IMAGE = TEST_DIR/'images'
TEST_LABELS = TEST_DIR/'labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE


### Training and Validation Dataset setup

In [8]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return tf.cast(image, tf.float32)


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    print(bbox)
    return {"images": image, "bounding_boxes": bounding_boxes}

In [9]:
import os

import numpy as np
import tqdm
from tqdm.notebook import tqdm


class PrepareDataset:
    def __init__(self, image_dir: str, label_dir: str) -> None:
        """
        Args:
            image_dir (str): Path to the directory containing the images.
            label_dir (str): Path to the directory containing the labels.
        """
        self.image_dir = image_dir
        self.label_dir = label_dir

    def get_dataset(self) -> tuple[list[str], list[int], list[np.ndarray]]:
        """Loads and parses YOLOv8 labels.

        Args:
            None

        Returns:
            tuple[list[str], list[int], list[np.ndarray]]: A tuple containing:
                - A list of image paths.
                - A list of class ids.
                - A list of bounding boxes, where each bounding box is an array of 8 floats.
        """
        image_paths = []
        class_ids = []
        bboxes = []
        for file_name in tqdm(os.listdir(self.image_dir)):
            if file_name.endswith((".jpg", ".png")):
                imaga_file_path = os.path.join(self.image_dir, file_name)
                label_file_path = os.path.join(self.label_dir, file_name.replace(
                    ".jpg", ".txt").replace(".png", ".txt"))

                if not os.path.exists(label_file_path):
                    print(f"Label file not found for image: {imaga_file_path}")
                    continue

                with open(label_file_path, 'r') as f:
                    lines = f.readlines()

                if not lines:
                    continue

                image_bboxes = []
                image_classes = []


                # 0 0.458736435546875 0.3806510419921875 0.3614540244140625 0.389472591796875 0.36892872265625 0.48237111328125 0.457112263671875 0.471341630859375 0.458736435546875 0.3806510419921875
                for line in lines:
                    try:
                        values = [float(value) for value in line.split()]
                        class_id = int(values[0])
                        coords = np.array(values[1:9]) # Extract the four corner points

                        # Reshape coords to (1, 4) if it's a single box
                        # if coords.size == 4:
                        #     coords = coords.reshape(1, 4)  # Ensure it's a 2D array even for one box
                                # Reshape coords to (4, 2) if it's a single box
                        if coords.size == 8:
                            coords = coords.reshape(4, 2)  # Ensure it's a 2D array with four points

                        # Calculate xmin, ymin, xmax, ymax
                        x_coords = coords[:, 0]
                        y_coords = coords[:, 1]
                        xmin = np.min(x_coords)
                        ymin = np.min(y_coords)
                        xmax = np.max(x_coords)
                        ymax = np.max(y_coords)
                        image_bboxes.append(np.array([[xmin, ymin, xmax, ymax]]))
                        image_classes.append(class_id)

                    except Exception as e:
                        print(f"[ERROR] - {e} in file {label_file_path} on line: {line}")
                        continue
                    
                image_paths.append(imaga_file_path)
                class_ids.append(image_classes)
                bboxes.append(np.array(image_bboxes))  # Concatenate boxes for the image
                
        # Ensure bounding boxes have the correct shape
        bboxes = [np.array(bbox).reshape(-1, 4) for bbox in bboxes]
        class_ids = [np.array(cls) for cls in class_ids]
        return image_paths, class_ids, bboxes


In [10]:
# from utils.prepare_dataset import PrepareDataset

preparer_train_ds = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR)
image_paths, class_ids, bboxes = preparer_train_ds.get_dataset()
len(image_paths), len(class_ids), len(bboxes)

  0%|          | 0/3631 [00:00<?, ?it/s]

(1804, 1804, 1804)

In [11]:
class_ids

[array([1]),
 array([6]),
 array([0]),
 array([6]),
 array([2]),
 array([0, 0]),
 array([0]),
 array([2]),
 array([6, 6]),
 array([1, 1]),
 array([6, 6]),
 array([5]),
 array([2]),
 array([1]),
 array([1]),
 array([2]),
 array([4]),
 array([0]),
 array([1, 1, 1]),
 array([1]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([1]),
 array([5, 5]),
 array([2]),
 array([1]),
 array([2, 2]),
 array([4]),
 array([2]),
 array([0]),
 array([5]),
 array([2]),
 array([6, 6]),
 array([4]),
 array([5]),
 array([2]),
 array([1, 1]),
 array([4, 3]),
 array([5]),
 array([2]),
 array([1]),
 array([1]),
 array([1]),
 array([5]),
 array([5]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([1]),
 array([4]),
 array([2]),
 array([4]),
 array([4]),
 array([0]),
 array([6]),
 array([0]),
 array([4]),
 array([4]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([2, 2]),
 array([4]),
 array([1]),
 array([5]),
 array([1, 1]),
 array([2]),
 array([0]),
 array([0, 0

In [12]:
bboxes

[array([[0.44046832, 0.31309316, 0.7659444 , 0.55647547]]),
 array([[0.2908256 , 0.38371963, 0.41078433, 0.51536671]]),
 array([[0.42242639, 0.42986278, 0.60367703, 0.54083247]]),
 array([[0.41856023, 0.40947881, 0.60555091, 0.4855385 ]]),
 array([[0.36529299, 0.396875  , 0.59382645, 0.55958793]]),
 array([[0.72946702, 0.36948838, 0.83719463, 0.49267923],
        [0.5997687 , 0.39877222, 0.69471286, 0.51465213]]),
 array([[0.55960587, 0.36421872, 0.74749557, 0.49238093]]),
 array([[0.57687778, 0.60538377, 0.68442293, 0.71111296]]),
 array([[0.38905194, 0.49061669, 0.71110769, 0.59193098],
        [0.70179982, 0.41440683, 0.83750585, 0.50227231]]),
 array([[0.54785422, 0.59742854, 0.76764881, 0.7081445 ],
        [0.40979874, 0.47174375, 0.60722812, 0.55683948]]),
 array([[0.55737016, 0.34697354, 0.72298033, 0.48769584],
        [0.28804224, 0.44486309, 0.71399357, 0.56016927]]),
 array([[0.63090101, 0.28415787, 0.90649764, 0.51598963]]),
 array([[0.40071612, 0.32856282, 0.54898276, 0.4

In [13]:
bboxes_tensor = tf.ragged.constant(bboxes)
classes_tensor = tf.ragged.constant(class_ids)
image_paths_tensor = tf.ragged.constant(image_paths)
bboxes_tensor, classes_tensor, image_paths_tensor

I0000 00:00:1739345679.405043    9487 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7279 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


(<tf.RaggedTensor [[[0.4404683160493827, 0.31309315625, 0.7659444000000001,
    0.556475474609375]]                                   ,
  [[0.2908256, 0.3837196328125, 0.4107843309523809, 0.515366712890625]],
  [[0.42242639346177135, 0.429862784787346, 0.6036770273964255,
    0.5408324747228431]]                                       , ...,
  [[0.3580157093596059, 0.236966337890625, 0.6372623522167488,
    0.489810345703125]]                                       ,
  [[0.3599481423220974, 0.63148377734375, 0.5786385131086142,
    0.736201748046875]]                                      ,
  [[0.40548455952380946, 0.317203783203125, 0.6002938095238095,
    0.403377921875]]                                           ]>,
 <tf.RaggedTensor [[1], [6], [0], ..., [0], [4], [1]]>,
 <tf.Tensor: shape=(1804,), dtype=string, numpy=
 array([b'datasets/BoneFractureYolo8/train//images/image1_337_png.rf.ea9e9cb65cf9948ca00b6cb07bd84603.jpg',
        b'datasets/BoneFractureYolo8/train//images/image1_118

In [14]:
train_data = tf.data.Dataset.from_tensor_slices(
    (image_paths_tensor, classes_tensor, bboxes_tensor))

In [15]:
train_data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), RaggedTensorSpec(TensorShape([None]), tf.int32, 0, tf.int64), RaggedTensorSpec(TensorShape([None, None]), tf.float64, 1, tf.int64))>

In [16]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(BATCH_SIZE * 4)\
    .ragged_batch(BATCH_SIZE, drop_remainder=True)


tf.RaggedTensor(values=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:1", shape=(None,), dtype=float64), row_splits=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:0", shape=(None,), dtype=int64))


In [17]:
for batch in train_ds.take(1):
    print("Before augmentation:")
    print("Images shape:", batch['images'].shape)
    print("Bounding boxes shape:", batch['bounding_boxes']['boxes'].shape)
    print("Classes shape:", batch['bounding_boxes']['classes'].shape)

Before augmentation:
Images shape: (16, None, None, 3)
Bounding boxes shape: (16, None, None)
Classes shape: (16, None)


2025-02-12 07:34:39.981830: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
train_ds = train_ds.map(resizing_layer, num_parallel_calls=tf.data.AUTOTUNE)
for batch in train_ds.take(1):
    print("After augmentation:")
    print("Images shape:", batch['images'].shape)
    print("Bounding boxes shape:", batch['bounding_boxes']['boxes'].shape)
    print("Classes shape:", batch['bounding_boxes']['classes'].shape)

NameError: name 'resizing_layer' is not defined

In [ ]:
inputs = next(iter(train_ds.take(2)))
inputs["bounding_boxes"]

: 

: 

: 

: 

In [ ]:
valid_datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
                                label_dir=VALID_LABELS_DIR)
valid_image_paths, valid_labels, valid_bboxes = valid_datasets.get_dataset()
len(valid_image_paths), len(valid_labels), len(valid_bboxes)

: 

: 

: 

: 

In [ ]:

val_data = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(valid_image_paths),
                                               tf.ragged.constant(valid_labels),
                                               tf.ragged.constant(valid_bboxes)))
val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

: 

: 

: 

: 

In [ ]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(IMG_SIZE, IMG_SIZE),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

: 

: 

: 

: 

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

: 

: 

: 

: 

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

: 

: 

: 

: 

In [ ]:
import sigstore
print(sigstore.__version__)

: 

: 

: 

: 

In [ ]:

backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)

: 

: 

: 

: 

In [ ]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)

: 

: 

: 

: 

: 

: 

: 

: 